In [26]:
import pandas as pd
from datetime import datetime, timedelta

In [27]:
transaction_df = pd.read_csv('app_transaction.csv', parse_dates=['timestamp'])
analytics_df = pd.read_csv('app_analytics.csv', parse_dates=['timestamp'])

In [28]:
now = datetime.now()
time_24h = now - timedelta(hours=24)
time_3d = now - timedelta(days=3)
time_7d = now - timedelta(days=7)

In [29]:
def aggregate_signals(df, value_col):
    # Filter separately by time window
    df_24h = df[df['timestamp'] >= time_24h]
    df_3d = df[df['timestamp'] >= time_3d]
    df_7d = df[df['timestamp'] >= time_7d]

    # Aggregate each
   
    agg_24h = df_24h.groupby("product_id")[value_col].sum().rename("24h")
    agg_3d = df_3d.groupby("product_id")[value_col].sum().rename("3d")
    agg_7d = df_7d.groupby("product_id")[value_col].sum().rename("7d")
    # total = df.groupby("product_id")[value_col].sum().rename("total")

    # Merge all
    agg = pd.concat([ agg_24h, agg_3d, agg_7d], axis=1).fillna(0).reset_index()
    return agg


In [30]:
sales_agg = aggregate_signals(transaction_df, "purchase_count")
sales_agg.columns = ['product_id', '24h_sales', '3d_sales', '7d_sales']


In [31]:
views_agg = aggregate_signals(analytics_df, "view_count")
views_agg.columns = ['product_id', '24h_views', '3d_views', '7d_views']


In [32]:
cart_agg = aggregate_signals(analytics_df, "addtocart_count")
cart_agg.columns = ['product_id', '24h_cart', '3d_cart', '7d_cart']


In [33]:
wish_agg = aggregate_signals(analytics_df, "wishlist_count")
wish_agg.columns = ['product_id', '24h_wish', '3d_wish', '7d_wish'] 

In [34]:
merged = sales_agg.merge(views_agg, on="product_id", how="left") \
                  .merge(cart_agg, on="product_id", how="left") \
                  .merge(wish_agg, on="product_id", how="left")

In [35]:
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

In [40]:
def calculate_trending_score(df,internal_weights,business_weights,threshold_value):
    for i in internal_weights:
        sub_weights=internal_weights[i]
        df[f"weighted_{i}"] = 0.0
        for key,value in sub_weights.items():
            df[f"weighted_{i}"]+=value*df[f"{key}_{i}"]

    df["trending_score"]=0.0
    for key,value in business_weights.items():
        df[f"norm_{key}"]=normalize(merged[f"weighted_{key}"])
        df[f"norm_with-{key}"]=value*df[f"norm_{key}"]
        df["trending_score"]+= df[f"norm_with-{key}"]

    df["is_trending"] = df["trending_score"] >= threshold_value
    top_trending = merged[merged["is_trending"]==True].sort_values(by="trending_score", ascending=False)
    top_trending.to_csv("trending_products_1.csv", index=False)


In [41]:
threshold_value=0.60
internal_weights={"sales":{"24h":0.6,"3d":0.3,"7d":0.1},
                 "views":{"24h":0.6,"3d":0.4,"7d":0.0},
                 "cart":{"24h":0.6,"3d":0.4,"7d":0.0},
                 "wish":{"24h":0.6,"3d":0.4,"7d":0.0}}
business_weights={"sales":0.50,"views":0.10,"cart":0.30,"wish":0.10}
calculate_trending_score(merged,internal_weights,business_weights,threshold_value)

In [42]:
print(merged[["weighted_views", "norm_views"]])
print(merged[["weighted_cart", "norm_cart"]])

    weighted_views  norm_views
0            201.2    0.454324
1            215.8    0.498782
2            138.4    0.263094
3            329.0    0.843484
4            113.6    0.187576
5            156.4    0.317905
6             52.0    0.000000
7            380.4    1.000000
8            201.0    0.453715
9            162.2    0.335566
10            80.0    0.085262
11           222.6    0.519488
12           115.6    0.193666
13            69.2    0.052375
14            65.2    0.040195
15           193.2    0.429963
16           229.0    0.538977
17           168.4    0.354446
18           249.2    0.600487
19           173.0    0.368453
    weighted_cart  norm_cart
0           141.4   0.756498
1            85.6   0.374829
2            41.4   0.072503
3           137.2   0.727770
4            55.2   0.166895
5            49.6   0.128591
6            56.0   0.172367
7           177.0   1.000000
8           123.4   0.633379
9            77.8   0.321477
10           46.0   0.103967
1